In [3]:
library(repr)
library(tidyverse)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [8]:
spotify_data <- read_csv("data/spotify_scraped_final.csv") |>
                select(-artists, -explicit, -id, -name, -playlist_subgenre) |>
                na.omit()
                    

# Split data (training 75% test 25%)
spotify_split <- initial_split(spotify_data, prop = 0.85, strata = playlist_genre)  
spotify_train <- training(spotify_split)   
spotify_test <- testing(spotify_split)

# Cross-Validation
spotify_vfold <- vfold_cv(spotify_train, v = 5, strata = playlist_genre)


Rows: 32850 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): artists, id, name, playlist_genre, playlist_subgenre
dbl (15): duration_ms, popularity, track_number, danceability, energy, key, ...
lgl  (1): explicit

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
# KNN Steps
k_vals <- tibble(neighbors = c(2, 3, 4, 5, 6))

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

spotify_recipe <- recipe(playlist_genre ~ . , data = spotify_train)

spotify_fit <- workflow() |>
                     add_recipe(spotify_recipe) |>
                     add_model(knn_spec) |>
                     tune_grid(resamples = spotify_vfold, grid = k_vals)